# Introduction to Gene Expression Analysis by Arrays - Prelab

### I.1. Overview and Objectives
In this lab, we will introduce you to the analysis of gene expression data, as generated by microarrays. After this prelab and some in-class activities, you will:

* Use R and the Bioconductor package to load libraries and tools required for the analysis of these data
* Read data and files into R that you will need for analysis: CEL files, and phenotypes
* Learn about several useful functions for viewing data: hist(), image(), boxplot()
* Learn about Normalization: what and why and an R function to normalize data: rma()
* Build a design module for the analysis of your data: model.matrix()
* Perform differential expression analysis: efit()
* Output results from your expression analysis: print.file
* Report p-values from the top results: topTable()

For those also interested, we include a couple of optional features which may be of interest:
* Visualize results of analysis via a Quantile-Quantile (QQ) plot.
* Another 'nifty' functions: heatmap, volcano plots.

In the following prelab, we're going to walk through a range of commands (and motivatory details) - don't worry too much about executing the code here (in the assignment, you'll be doing a bit more heavy lifting!).  

### I.2. A brief note on sageworkbooks

Until now, you've been working with ipython notebooks. In the next two modules, you will be using sageworksheets. In general, these are not altogether dissimilar to ipython notebooks - and for practical purposes for you, should be treated as such.

* To exceute the contents of a given set of commands or execute a block of code: as before, use shift-return
* There will be places where we ask you to actually write answers in the cell. Do just this - and no need to execute code from them.
* You'll see a green flashing light as the code is running. If it is blinking, it means code is running. 
* If you were expecting output, but none is run, you can always click on the "restart" button. However, if you do this, you will need to execute the previous lines of code, if the current part that you are working on depends on the previous code.
* Try to avoid clicking on the bars that separate different parts of code.
* If you run into trouble where your code or comments seems to dissapear, try to double click on the Output box (highlighted in green). Otherwise, ask a prof or TA for help.


### II. Brief review: Technology for Gene Expression Analysis

It turns out that there are a wide range of technologies used to assay gene expression in the genome. A short review of the types of microarray technologies are given in the first part of a paper by Miller and Tang (2009). (see paper in Canvas)

But not only do the types of technology used to assay gene expression differ, the types of transcripts that are assayed by a given microarray will also differ:
* One might want to use a small collection of probes to query all transcripts. 
* Or, probes for every exon in all transcripts. 
* Or alternatively still, only a subset of all transcripts (ie. 'onco-chips' which select only cancer-related oncogenes or tumor supressor genes). 
* And finally, the probes one designs for a mouse will be different for that for humans.

<u>__It is important you know what type of array experiment design you are performing, or analyzing__</u>. Specifically, this has importantly implications for pathway enrichment analysis. But more practically, you need to know what _annotation files_ to use, so that you can correctly map the probe intesnity reads to transcripts to genes.

### III. Brief review: Motivation for Gene Expression Studies

If this is a new scientific area to you, we've provided a couple of review papers to you to read:

(1) a short PloS Comp Biology paper from Slonim and Yanai (2009). <br \>
(2) a longer review paper from Alison et al. (2006).

These papers can be found on Canvas. Broadly, I have summarized the following take home messages:

* Gene expression microarrays allow the survey of the expression of virtual all genes in the genome, at a specfic point in time. Often, in different treated or untreated states.<br \>
* One objective of these experiments is to determine if the expression of a given gene is different between treatments, conditions, or tissues: i.e.<br \>
H0 = The condition has no effect on the expression of a gene (β = 0) <br \>
H1 = The condition has an effect on the expression of a gene (β != 0) <br \>
and our goal is to determine if β is significantly different from zero. 
* Here, note that we are describing the hypothesis test for a __single__ gene. In a single microarray experiment, we assay thousands of genes, simultaneously. As a result, **_mutliple-hypothesis testing issues apply_**. <br \> 
In other words, we should not consider all genes that have a nominal P < 0.05 to "significantly differ" in levels of gene expression (e.g. if 1000 genes were tested, we'd expect 50 by chance to score P < 0.05. Not what we want!). The levels of significance require burden of proof given the number of genes (and hypotheses) tested. There are ways apply correction for mutliple testing, and we'll go over some of that later.
* A corollary to this question is to identify specific sets of genes (or pathways) that differ between these groups; typically, we think about the use of gene sets from Gene Ontology or curated sets of Pathways to test these hypotheses.
* The design of an experiment follows this general workflow:

![Figure 1 from Slonim and Yanai, 2009](img/Slonim_fig1-v2.png "Figure 1 from Slonim and Yanai, 2009")

* We are going to focus our modules here on the use of Affymetrix microarrays. The major file format by which data is provided by these arrays are files in the .CEL format. The __CEL file__ is a file that summarizes of the results of the intensity calculations on the pixel values read from the microarray. These files contain a bunch of information how how the intensity value for the given probe was calculated, and summary statistics for that intensity (mean, standard deviation), in addition to perhaps other pieces of information about the experiment performed. <br \> Unfortunately, these files are quite large and, consequently, are stored in a binary (not-human readable) format: if you try to 'peek' at them in UNIX (using less, or more, or head), you'll probably get a bunch of jibberish. Note that there are ways to convert a CEL file into a human readable format, but we won't go into those details here.

### IV. Finding Data: the Gene Expression Omnibus (GEO) catalog

There is a huge trove of expression data sets to be found at [GEO](http://www.ncbi.nlm.nih.gov/geo/). These data are nicely annotated, organized, and easily searchable. Take a moment to search around this index, search for your favorite terms, see what you can find!

To look at one example data set, take: GSE47516

If we search for this term, we get a lot of specific information about the experiment, the samples that were assayed, and the platform that was used. In this case, 3 groups of n=7 samples, related to cerebellum were collected on the Affymetrix Mouse 430 2.0 Array. 

You will also notice a link to a file archive (.tar) which contains all of the CEL data that is referred to here. If you want, this is a place where you can download data. However, if we know the GSE number, we can use R to download the file directly from GEO (see below).

### IV. Bioconductor: The Veritable "Swiss-Army" Knife for High-throughput Data Analysis

After a great deal of work from a large community of bioinformaticians, there are a number of tools available to you which can be used for the analysis of microarray gene expression data. Many of these tools are housed in a vault called [Bioconductor](http://bioconductor.org). Bioconductor makes all of these tools available to you, via R:

    source("https://bioconductor.org/biocLite.R")
    biocLite()

Bioconductor contains libraries for tools to load and process data, to normalize, make plots, visualize, perform analysis, annotations, and more. If there is something you want to do with your array data, chances are there is a package (and a function) somewhere in this repository. The website has lots of searchable packages, so that's a great place to start. <br \>

With this introduction in hand, we are now ready to 'wade' into the various packages, functions, and procedures you will use for a gene expression analysis.

### V.1. Analysis Pipeline: Packages in R: How, where, what?

In this section, we'll review how to load packages in R, and specifically, list some of the key package you will need to load when performing a differential gene-expression microarray study.

For sanity, I like to make sure my packages are loaded from bioconductor, then loaded into R. There are two packages which you will almost always want to load: 
* _GEOquery_: a package to automatically obtain data from the Gene Expression Omnibus
* _oligo_: a package to analyze oligonucleotide arrays (expression/SNP/tiling/exon) at probe-level. 
* _limma_: Data analysis, linear models and differential expression for microarray data.

First, we'll initialize libraries we need\* from bioconductor using the function biocLite():

    ## this is the section for drawing packages from Bioconductor
    source("https://bioconductor.org/biocLite.R")
    biocLite("GEOquery")

\*In situations where you are loading these packages for the first time on your own computer, you may have to install the oligo, limma, gcrma using the biocLite() function: 
   
    biocLite("oligo") 

However, on cloud.sagemath.com, we have installed this for you already, so you only need to call the library() function.

Next, we'll load the libraries into R using library():

    ## this is the section for loading packages in R
    library(GEOquery)
    library(oligo)
    library(limma)

To keep code organized, I like to place all of the packages I load at the _beginning_ of my R pipeline. Once my packages are loaded, then proceed subsequently to perform analysis steps. In that way, my workspace is "initialized" and all I need do it copy and paste the library/package initialization steps when running my R pipeline.

Next, given the array we are analyzing, we'll want to load packages that refer to the specific annotation of the array we are analyzing. __This is where you need to know what type of array and organism you are analyzing__. For example, if we used the Affymetrix "Mouse Gene Array 2.1 ST" array, we could use the following:

    ## this is the section where I draw annotation packages for my array from bioconductor
    biocLite(pd.mogene.2.1.st)
    
    ## this is the section where I load annotation packages into R
    library(pd.mogene.2.1.st)

The annotation files you will require depend on the particulars of your experiment: what organism, what version of array, what type of experiment. <br \>
**Don't guess here!**<br \> 
Your annotation files must be the same as what is expected for your array. In the pipeline below, one of the functions will try to be 'smart' and detect the library that it thinks it should use for your data, and will install the correct annotation libraries for you. However, if you see that this is not correct, you will need to adjust accordingly.

A list of annotation packages can be found [here](http://bioconductor.org/packages/release/BiocViews.html#___AnnotationData).

Now that we have the GEOquery package loaded, let's imagine we wanted to download a set of data from the GEO repository, rather than copy and upload directly. For our GSE47516 data set, we can use:

    getGEOSuppFiles("GSE47516")
    
This will create a directory (named the data set we downloaded); inside, there will be the .tar archive file we requested. We'll have to use UNIX to untar the file, and then to gunzip the file contents:

    cd GSE47516
    tar -xvf GSE47516_RAW.tar
    gunzip *.gz

### V.2. Analysis Pipeline: The phenotype data file, loading into R

You will need to prepare a file which connects each of the CEL files that you will analyze to a treatment condition that you want to compare. An easy way to make this file is to create a comma separated file with the sample and the condition. In the example described in this prelab, we have data from 4 Liver and 4 Spleens from mouse, and we want to compare gene expression of. The bit of code should look familar to you:

In [1]:
pData = open('example-pheno.csv', 'r')
for line in pData:
    line = line.rstrip('\n\r')
    print line

Sample,Tissue
MoGene-2_1_GA_Liver_1_C01.ga.cel,Liver
MoGene-2_1_GA_Liver_A01.ga.cel,Liver
MoGene-2_1_GA_Liver_C01.ga.cel,Liver
MoGene-2_1_GA_Liver_G01.ga.cel,Liver
MoGene-2_1_GA_Spleen_A01.ga.cel,Spleen
MoGene-2_1_GA_Spleen_C01.ga.cel,Spleen
MoGene-2_1_GA_Spleen_E01.ga.cel,Spleen
MoGene-2_1_GA_Spleen_G01.ga.cel,Spleen


Now, let's load the phenotype file into R using

    phenoData <- read.AnnotatedDataFrame("example-pheno.csv", header=TRUE, sep=",")
    
This will create an object (pData) that effectively stores the information from our phenotype file. Note the argument include a header (=True), and that the content of the file are <u>sep</u>arated by a comma.

we can get a little summary of that data to check that it's loaded as intended in R via the following

> \> pData(phenoData) <br \>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Tissue <br \>
MoGene-2_1_GA_Liver_1_C01.ga.cel  Liver <br \>
MoGene-2_1_GA_Liver_A01.ga.cel    Liver <br \>
MoGene-2_1_GA_Liver_C01.ga.cel    Liver <br \>
MoGene-2_1_GA_Liver_G01.ga.cel    Liver <br \>
MoGene-2_1_GA_Spleen_A01.ga.cel  Spleen <br \>
MoGene-2_1_GA_Spleen_C01.ga.cel  Spleen <br \>
MoGene-2_1_GA_Spleen_E01.ga.cel  Spleen <br \>
MoGene-2_1_GA_Spleen_G01.ga.cel  Spleen

Here, we can see that the phenotype file has 8 cel files, one set where the Tissue is liver, the other Spleen.

### V.3. Analysis Pipeline: Loading cel file data into R

Now, we'll want to load the expression array (CEL) files into R for analysis. I like to get my data organized so that I don't have piles of array data strewn about, so typically I make a directory where all of these data are stored. Here, we have a directory called 

    cel-data/

that contains all of our cel files. 

Let's get a list of the files that are housed in that directory:

    celFiles <- list.celfiles("cel-data",full.name=T)
    
> \> celFiles <br \>
[1] "cel-data/MoGene-2_1_GA_Liver_1_C01.ga.cel" <br \>
[2] "cel-data/MoGene-2_1_GA_Liver_A01.ga.cel" <br \>
[3] "cel-data/MoGene-2_1_GA_Liver_C01.ga.cel" <br \>
[4] "cel-data/MoGene-2_1_GA_Liver_G01.ga.cel" <br \>
[5] "cel-data/MoGene-2_1_GA_Spleen_A01.ga.cel" <br \>
[6] "cel-data/MoGene-2_1_GA_Spleen_C01.ga.cel" <br \>
[7] "cel-data/MoGene-2_1_GA_Spleen_E01.ga.cel" <br \>
[8] "cel-data/MoGene-2_1_GA_Spleen_G01.ga.cel" <br \>

Ok, now let's read those data into R using the read.celfiles() command

    affyRaw <- read.celfiles(celFiles)

At this point, R will try to identify the proper annotation library for you. In this case, the function correctly identified these data as Affymetrix 2.1 Gene ST array data from mouse, and is using this set of annotations

    pd.mogene.2.1.st
    

### V.4. Analysis Pipeline: Some visualization functions

After loading the CEL data into R, it is often useful to look at high level summaries and plots of the data, before we get down to performing analysis. In this prelab, we are going to walk through a couple of basic functions, **but not an exhaustive listing of them**. For your own data and analysis, you will want to explore the data in much more detail, check outliers, etc.

A [user's guide for the oligo() package](http://bioconductor.org/packages/devel/bioc/vignettes/oligo/inst/doc/oug.pdf) contains a large number of useful details which could be explored for more in depth purposes.

#### hist() and boxplot()

These functions allow you to visualize the distribution of intensities of the probes assayed in each CEL file:

    boxplot(affyRaw, which=c("all"))
    hist(affyRaw, which=c("all"))

will generate the following two plots:

![example-boxplot](img/ex_boxplot.png "Example Box Plot for Probe Intensity Data")

![example-hist](img/ex_hist.png "Example histogram for Probe Intensity Data")

#### image()

Sometimes there are spatial feature on the arrays, and you might want to visually inspect your arrays.

    image(affyRaw[,1])
    

![example-image](img/ex_image.png "Example histogram for Probe Intensity Data")

### V.5. Analysis Pipeline: Normalization

As we can see from the histogram or box plots, the distribution of intensity values differs across each CEL file. This is problematic, because we want intensity data to be comparable across different arrays (and across treatment conditions). To deal with this issue, we need to _normalize_ the data in some way. As it turns out, there are many ways to normalize data, and many packages exist that implement a wide range of normalization procedures. 

One robust way to normalize data is using the Robust Multichip Average (RMA) approach. To perform normalization using RMA, simply use:

    genenorm <- rma(affyRaw)

These look much better, don't they?

![example-boxplot-norm](img/ex_boxplot_norm.png "Example Box Plot for Probe Intensity Data (Normalized)")

![example-hist-norm](img/ex_hist_norm.png "Example Histogram for Probe Intensity Data (Normalized)")

### V.6. Analysis Pipeline: Design Matrix for Analysis

Next up, we need to create a matrix that tells R what groups to compare against. In this example, we will design a simple, two group comparison (Liver vs. Spleen):

    group <- factor(t(pData(phenoData)[,1]))
    
> \> group
[1] Liver  Liver  Liver  Liver  Spleen Spleen Spleen Spleen

    design <- model.matrix(~group,pData(phenoData))

> \> design <br \>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;(Intercept) GroupSpleen <br \>
MoGene-2_1_GA_Liver_1_C01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;0 <br \>
MoGene-2_1_GA_Liver_A01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;0 <br \>
MoGene-2_1_GA_Liver_C01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;0 <br \>
MoGene-2_1_GA_Liver_G01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;0 <br \>
MoGene-2_1_GA_Spleen_A01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;1 <br \>
MoGene-2_1_GA_Spleen_C01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;1 <br \>
MoGene-2_1_GA_Spleen_E01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;1 <br \>
MoGene-2_1_GA_Spleen_G01.ga.cel&emsp;&emsp;&emsp;&emsp;&emsp;1&emsp;&emsp;&emsp;&emsp;&emsp;1 <br \>
attr(,"assign") <br \>
[1] 0 1 <br \>
attr(,"contrasts") <br \>
attr(,"contrasts")$Group <br \>
[1] "contr.treatment" <br \>

As you can see, all of the CEL files that are Liver are set to "0", and Spleen "1", and the appropriate object has been created. Here, we will measure expression in Spleen (=1), relative to Liver (=0).

### V.6. Analysis Pipeline: Performing differential expression analysis and Summarizing Results

Now that we have our design matrix in hand, we can now prepare to test our probes for differences in gene expression. To perform this analysis, we will use linear regression to determine if the intensity of the probe is correlated with the effect the treatment (in this case, tissue), and if so, estimate the effect of the probe intensity (β). Because the number of arrays we typically analyze is small, we need to use _moderated_ t-statistics and an empirical Bayes procedure, equivalent to shrinkage of the estimated sample variances of the probe intensities towards a pooled estimate, resulting better stability for inference (see [Smith 2004](http://www.statsci.org/smyth/pubs/ebayes.pdf) for more details).

    fit <- lmFit(genenorm,design)
    efit <- eBayes(fit)
    
The result is a large table of scores! We can use another function, topTable(), to obtain results

    tt <- topTable(efit, sort="P", n=500)

Here, I have opted to sort the Table by P-value, and am only recording the top 500 results in the scan. If I only wanted a table with all the results, I could set n=Inf instead, but keep in mind that will generate a VERY large file (>200 Mb).
    
> \> tt[1:5,] <br \>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;logFC  AveExpr&emsp;&emsp;&emsp;&emsp;&emsp;t&emsp;&emsp;&emsp;P.Value&emsp;&emsp;&ensp;adj.P.Val&emsp;&emsp;&emsp;&ensp;B <br \>
17405908 -4.136117 6.911107 -67.54411 5.597037e-13 1.415179e-08 20.03929 <br \>
17284936 -4.116741 6.375286 -63.26197 9.795902e-13 1.415179e-08 19.60969 <br \>
17419622 -3.586706 7.065164 -61.67103 1.217751e-12 1.415179e-08 19.43799 <br \>
17472364 -4.903165 5.905147 -59.14892 1.739743e-12 1.415179e-08 19.15110 <br \>
17219356 -3.681500 7.225212 -58.70824 1.854509e-12 1.415179e-08 19.09902 <br \>

Not surprisingly, there are several probes which are differently expressed between liver and spleen, in this case, expressed lower in _Spleen_ relative to Liver (note the negative values here). Positive scores here would indicate higher expression in Spleen, relative to Liver.

### V.6. Analysis Pipeline: Output results to file

Often, we will want to output our results to a file, so that we can summarize, save results, and extract information more readily using other tools. We can use write.table for this:

    write.table(tt,file="example_allresults.txt",quote=F,sep=",",row.names=T,col.names=T);
    
Note that the header in the file will be off-by-one column, e.g. the first column is not logFC, but the probeID.

Also note here that in this case, we're separating the results by comma (sep=","), which generally should be fine. In some cases, you may have a table that contains not just numbers, but also words or descriptions (which may themselves include whitespaces, or even commas!).

If that is the case, you can set quote=T, which will keep those units together. (parsing those data files in UNIX can be a trick issue, however).

### Bonus material

There are a cople of other nice function we might care about plotting.

Volcano plot: these help us visualize the distribution, at a high level, of the significance of association by the fold change in expression. 

    volcanoplot(efit,coef=2)

![example-volcano](img/ex_volcano.png "Example Volcano Plot: Spleen vs. Liver")

QQ-plot: This plot is designed to help determine if the distribution of association statistics is reasonably well calibrated. In general, for most experiments, we expect only a handful of positive findings; most results will not show differences in probe intensity across treatment (i.e. are "null" distributed). If all results were, in fact, null, and there were no excess number of significant associations, the scores of our statistics should line up on the x=y line.

    qqt(efit$t[,2],df=efit$df.residual+efit$df.prior)
    abline(0,1)

Outputting Expression values: In many cases, you may want to store or extract the expression values for your normalized data. You can use exprs() to do this (here, I'm reporting the first 100 probes)

    exp_data <- exprs(genenorm[1:100,])
    
and then write data to file

    write.table(exp_data,file="example_expr_output.txt",quote=F,sep=" ",row.names=T,col.names=T);

Heatmap: These plots allow you to visualize see larger patterns of co-expression across treatment groups. For example, to generate a heatmap for using the first 100 probes from your normalized gene expression data, you could use:

     #add me to libary steps
     source("https://bioconductor.org/biocLite.R")
     biocLite("Heatplus")
     library(Heatplus)
     
     reg1 <- regHeatmap(exprs(genenorm[1:100,]))
     plot(reg1)
     
There are a lot of different kinds of options for heatmaps. See the [Heatplus user guide](https://www.bioconductor.org/packages/3.3/bioc/vignettes/Heatplus/inst/doc/annHeatmap.pdf) for more details with examples.